In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Задание 10. Контест. Spotify recommendations

В этом задании вам предложен набор данных, содержащий информацию о некоторых плейлистах и треках музыкального сервиса Spotify.

Признаковое описание состоит из результатов некоторых запросов в Spotify Web API (в частности **GET https:// api.spotify.com/v1/audio-features/{id}** и **GET https:// api.spotify.com/v1/playlists/{playlist_id}/tracks**), которые возвращают объекты, содержащие различные свойства треков и плейлистов. В выборку также добавлены закодированные номера артистов и альбомов (сквозная нумерация).



1. В файле **added_info.csv** приводятся данные о том, когда и каким пользователем трек был добавлен в плейлист.

2. В файле **audio_features.csv** содержатся сведения о некоторых музыкальных свойствах треков, таких как танцевальность, акустичность, громкость, темп и других.

3. В файле **tracks_info.csv** содержатся данные об исполнителях и альбомах, связанных с той или иной композицией.


Более подробно про признаки можно прочитать в [документации (англ.)](https://developer.spotify.com/documentation/web-api) Spotify Web API.


В качестве метрики качества используется **F1-мера (F1 Score)** -- среднее гармоническое точности (Precision) и полноты (Recall).

В качестве значения, которое нужно предсказать, выступает бинарный признак принадлежности трека плейлисту. Если для пары плейлист-трек он равен 1, то плейлист содержит этот трек; если 0 -- то не содержит.

Вам необходимо разработать модель машинного обучения для предсказания **принадлежности треков плейлистам**, основанную на свойствах плейлистов, треков и пользователей.

В папке с заданием помимо этого и вышеописанных файлов находятся:
- 2 файла с данными об плейлистах и треках -- **train.csv** и **test.csv**, которые содержат, соответственно, тренировочную выборку (с известными значениями целевой переменной в поле **exists**) и признаковую часть тестовой выборки.

В качестве ответа на это задание вы должны предоставить **Kaggle-ноутбук** (как создать такой ноутбук, читайте ниже), который:
1. генерирует на выходе **csv-файл** со столбцом предсказанных принадлежностей треков плейлистам для пар из тестовой выборки и отправляет его в систему Kaggle. Пример такого файла находится в папке с заданием (**sample_submission.csv**);
2. разрешает чтение пользователю [Sergey Serov](https://www.kaggle.com/ssserov/account).

**НЕВЫПОЛНЕНИЕ ЛЮБОГО ИЗ УКАЗАННЫХ ПУНКТОВ ПРИВЕДЁТ К ОЦЕНИВАНИЮ ЗАДАНИЯ В 0 БАЛЛОВ!**

**Как создать и отправить корректный Kaggle-ноутбук:**

1. На странице соревнования перейдите на вкладку **Code** и нажмите **New Notebook**.
![kaggle_notebook_1](https://i.ibb.co/4N7Ncf1/kaggle-notebook-1.png)
2. **Никакие дополнительные данные для выполнения задания загружать не нужно** (но это не запрещено). Путь, по которому автоматически находятся необходимые файлы с данными, можно посмотреть, выполнив первую ячейку и изучив ее вывод.
![kaggle_notebook_2](https://i.ibb.co/q0bpTNd/kaggle-notebook-2.png)

3. Дать права на чтение ноутбука пользователю [Sergey Serov](https://www.kaggle.com/ssserov/account). Для этого в верхней панели ноутбука нужно нажать кнопку **Share**, далее выбрать **Add collaborators** и в поиске найти пользователя Sergey Serov (вместо owner будет написано collaborator). Не забудьте сохранить изменения кнопкой **Save**.
![kaggle_notebook_3.PNG](https://i.ibb.co/F0gRm3S/kaggle-notebook-3.png)
![kaggle_notebook_4.PNG](https://i.ibb.co/25mBCKb/kaggle-notebook-4.png)

После правильного действия Вы увидите:

![kaggle_notebook_5.jpg](https://i.ibb.co/J5Kbgk2/kaggle-notebook-5.jpg)

4. Для того, чтобы предсказания, полученные kaggle-ноутбуком были корректно учтены системой, он должен сохранять их следующей командой **submission.to_csv("/kaggle/working/submission.csv", index_label="index")**, где **submission** -- ваш датафрейм с предсказаниями (как в примере ниже).

5. Для отправки ноутбука в правой его панели выберите вкладку **Competitions**, нажмите на кнопку **Submit**, по желанию введите название и описание посылки и подтвердите нажатием кнопки **Submit**.

![kaggle_notebook_6.PNG](https://i.ibb.co/N2KsC0Y/kaggle-notebook-6.png)

6. Также этот ноутбук доступен в виде публичного кернела во вкладке **[Code](https://www.kaggle.com/competitions/cmc-ml-steam-activity-prediction/code)** соревнования. Его можно открыть, а затем сразу преобразовать в свой Kaggle-кернел, нажав на три точки справа вверху и выбрав опцию "Copy & edit notebook".
![kaggle_notebook_7.png](https://i.ibb.co/hVRWzmN/kaggle-notebook-7.png)

**Далее в этом ноутбуке покажем пример формирования csv-файла с предсказаниями принадлежности треков плейлистам для пар из тестовой выборки.**

Для начала импортируем библиотеки и загрузим данные из файлов **train.csv** и **test.csv**.

In [3]:
import pandas as pd
from sklearn.metrics import f1_score

In [4]:
# Используйте эти пути для запуска ноутбука на Kaggle
PATH_TO_KAGGLE_TRAIN = "D:/Programming/Programs_Py/KAGGLE/CMC_MSU_SPOTIFY/train.csv"
PATH_TO_KAGGLE_TEST = "D:/Programming/Programs_Py/KAGGLE/CMC_MSU_SPOTIFY/test.csv"
PATH_TO_KAGGLE_SUBMISSION = "D:/Programming/Programs_Py/KAGGLE/CMC_MSU_SPOTIFY/submission.csv"

# Если запускаете на своем компьютере, то впишите актуальные пути
# PATH_TO_TRAIN = # Your Path
# PATH_TO_TEST = # Your Path
# PATH_TO_SUBMISSION = # Your Path

train = pd.read_csv(PATH_TO_KAGGLE_TRAIN)
test = pd.read_csv(PATH_TO_KAGGLE_TEST)

X_train = train.drop(["index", "exists"], axis=1)
y_train = train["exists"]

train.head()

,index,playlist_id,track_id,exists
0,1050194,32931,07vS8obfeZbr8H4MgQfXR7,1
1,645550,35527,1PEqh7awkpuepLBSq8ZwqD,1
2,2630414,12203,7E8pPgBY84oDaXRcqODavR,0
3,3168314,21850,0Atml4huw4Fgyk6YSHiK4M,0
4,3144464,21391,4WYDmIZrwxBHdBYdvi5oQO,0


Затем создадим модель и обучим ее на тренировочной выборке.

In [72]:
import catboost
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import f1_score

In [117]:
splitter = ShuffleSplit(n_splits=1, train_size=0.95, test_size=0.05, random_state=42)
fitting_idx, test_idx = list(splitter.split(X_train))[0]
train_idx, validation_idx = list(splitter.split(train_idx))[0]
fit_X = X_train.iloc[train_idx]
fit_y = y_train.iloc[train_idx]
validation_X = X_train.iloc[validation_idx]
validation_y = y_train.iloc[validation_idx]
test_X = X_train.iloc[test_idx]
test_y = y_train.iloc[test_idx]

In [119]:
model = catboost.CatBoostClassifier(iterations=100, custom_loss='F1',
                                    cat_features=['track_id'])
model.fit(fit_X, fit_y, eval_set=(validation_X, validation_y))

Learning rate set to 0.5
0:	learn: 0.6519288	test: 0.6331292	best: 0.6331292 (0)	total: 845ms	remaining: 1m 23s
1:	learn: 0.6387533	test: 0.6148763	best: 0.6148763 (1)	total: 1.16s	remaining: 57s
2:	learn: 0.6285837	test: 0.6070038	best: 0.6070038 (2)	total: 1.51s	remaining: 48.9s
3:	learn: 0.6248454	test: 0.6040470	best: 0.6040470 (3)	total: 1.82s	remaining: 43.8s
4:	learn: 0.6232279	test: 0.6025191	best: 0.6025191 (4)	total: 2.09s	remaining: 39.7s
5:	learn: 0.6223838	test: 0.6010789	best: 0.6010789 (5)	total: 2.42s	remaining: 37.9s
6:	learn: 0.6220099	test: 0.6007061	best: 0.6007061 (6)	total: 2.68s	remaining: 35.6s
7:	learn: 0.6218121	test: 0.6003117	best: 0.6003117 (7)	total: 2.94s	remaining: 33.9s
8:	learn: 0.6217345	test: 0.6003346	best: 0.6003117 (7)	total: 3.2s	remaining: 32.4s
9:	learn: 0.6216622	test: 0.6002251	best: 0.6002251 (9)	total: 3.47s	remaining: 31.2s
10:	learn: 0.6215905	test: 0.5999745	best: 0.5999745 (10)	total: 3.72s	remaining: 30.1s
11:	learn: 0.6215689	test: 0.

Вычислим ошибку модели на тренировочной выборке.

In [121]:
y_pred = model.predict(test_X)

print(f"Train F1-score: {f1_score(test_y, y_pred)}")

Train F1-score: 0.6307717493387098


В заключение получим столбец предсказаний принадлежностей треков плейлистам для тестовой выборки и сохраним его в виде csv-файла (обратите внимание, что в выходном файле должно быть два столбца -- **index** и **exists**).

In [122]:
test_pred = model.predict(test.drop("index", axis=1))

submission = pd.DataFrame({"index": test["index"], 
                           "exists": test_pred})

In [123]:
submission.to_csv(PATH_TO_KAGGLE_SUBMISSION, index=False)